In [6]:
import os
import cv2
import imgaug.augmenters as iaa

# Set the input directory
dir_path = './'

# Define the prefix of the folders you want to process
folder_prefix = 'MDA231'

# Define the background color
background_color = (255, 255, 255)  # White color

# Define the morphological operation kernel size
kernel_size = (5, 5)

# Define the augmentation pipeline for PC3 images
augmentation_pipeline = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip with a probability of 0.5
    iaa.GaussianBlur(sigma=(0, 1.0)),  # Gaussian blur with a sigma between 0 and 1.0
    iaa.Affine(rotate=(-45, 45))  # Rotation between -45 and 45 degrees
    # Add more augmentation techniques as needed
])

# Iterate over the directories in the input directory
for entry in os.scandir(dir_path):
    if entry.is_dir() and entry.name.startswith(folder_prefix):
        # Get the folder name
        folder_name = entry.name

        # Set the input and output directories for the current folder
        input_dir = os.path.join(dir_path, folder_name)  # Directory containing the original cell images
        output_dir = os.path.join(dir_path, f"augmented_{folder_name}")  # Directory to store the augmented cell images

        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Get the list of cell images in the input directory
        cell_images = os.listdir(input_dir)

        #define number of samples (total for upsampling)
        desired_cell_count = 500  # Desired number of cell images

        # Compute the number of augmentation iterations required
        iterations = desired_cell_count // len(cell_images)
        left = desired_cell_count - (iterations *len(cell_images))
        print(left)

        # Augment the cell images and save them to the output directory
        augmented_count = 0  # Counter for the number of augmented images created
        while augmented_count < 500:
            for image_name in cell_images:
                if augmented_count >= 500:
                    break

                # Load the cell image
                image_path = os.path.join(input_dir, image_name)
                image = cv2.imread(image_path)

                # Augment the image
                augmented_images = augmentation_pipeline.augment_images([image])

                # Save the augmented images to the output directory
                for i, augmented_image in enumerate(augmented_images):
                    if augmented_count >= 500:
                        break

                    # Convert the augmented image to grayscale
                    gray = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2GRAY)

                    # Threshold the grayscale image to create a binary mask of the background regions
                    _, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV)

                    # Perform morphological operations to fill in gaps and smooth the boundary regions
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
                    mask = cv2.dilate(mask, kernel, iterations=2)
                    mask = cv2.erode(mask, kernel, iterations=1)

                    # Convert the binary mask to a 3-channel mask
                    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

                    # Invert the mask to represent the background regions
                    background_mask = cv2.bitwise_not(mask)

                    # Set the background regions to the desired background color
                    augmented_image = cv2.bitwise_and(augmented_image, background_mask) + cv2.bitwise_and(background_color, mask)

                    output_image_name = f"{image_name}_{augmented_count + 1}.png"
                    output_image_path = os.path.join(output_dir, output_image_name)
                    cv2.imwrite(output_image_path, augmented_image)

                    augmented_count += 1

20
6
18
20
7
235
41
32
24
16
62
24
59
89
20
12
102
14
